In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elrii\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
input_dir = r'./input/'
output_dir = r'./output/'

In [10]:
dfa = pd.read_sas(input_dir+'AUX_E_2007.XPT')
dfq = pd.read_sas(input_dir+'AUQ_E_2007.XPT')
dfd = pd.read_sas(input_dir+'DEMO_E_2007.XPT') #CHANGE

In [11]:
#dfq.to_excel(output_dir+"hi.xlsx")

In [12]:
dfa = dfa[['SEQN', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']].replace(666,100)

audiogram_col = ['AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']
audiogram_right = audiogram_col[:7]
audiogram_left = audiogram_col[7:]

dfa = dfa[~dfa[audiogram_col].isin([888]).any(axis=1)].dropna().astype(int)
dfa = dfa[(dfa[audiogram_right].sum(axis=1) >= 140) & (dfa[audiogram_right].sum(axis=1) >= 140)].reset_index(drop=True)
dfa

,SEQN,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,AUXU8KR,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL
0,42139,35,30,25,35,45,50,40,15,20,25,35,20,20,35
1,42386,30,30,25,35,30,20,35,30,35,20,40,30,15,15
2,42538,30,25,20,25,20,15,15,20,15,10,15,10,5,10
3,42956,45,35,30,45,45,70,65,10,10,10,10,10,10,0
4,43021,20,10,20,20,30,30,10,25,15,25,20,35,65,55
5,43148,30,15,15,20,20,45,30,30,30,30,15,45,35,35
6,43423,15,10,10,25,20,40,30,5,0,5,10,10,10,10
7,43439,10,20,25,30,25,35,30,0,10,15,20,25,10,0
8,43518,20,10,5,10,35,55,35,10,10,5,45,40,50,50
9,43721,75,75,65,60,65,85,75,20,15,20,5,5,50,30


In [13]:
dfq = dfq[['SEQN','AUQ131', 'AUQ260', 'AUQ270', 'AUQ290']] #CHECK
dfq=dfq.dropna(how='any').reset_index(drop=True)
dfq.shape

(88, 5)

In [14]:
dfd = dfd[['SEQN', 'RIAGENDR','RIDAGEYR', 'DMQMILIT']] #1 = male, 2 = female

thresholds = [0, 25, 35, 45, 55, dfd['RIDAGEYR'].max()] 
labels = [1, 2, 3,4,5]
dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'], 
                         bins=thresholds, labels=labels, right=True)
dfd = dfd.dropna(subset=['DMQMILIT']).reset_index(drop=True)
dfd.shape

(6371, 4)

In [15]:
dfV = pd.merge(pd.merge(dfd, dfa, on='SEQN'), dfq).astype(int)
print(dfV.shape)
dfV.to_excel(output_dir+"NHANES_2007-2008.xlsx") #CHANGE

(3, 22)


In [16]:
dfV

,SEQN,RIAGENDR,RIDAGEYR,DMQMILIT,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,...,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL,AUQ131,AUQ260,AUQ270,AUQ290
0,42956,2,1,2,45,35,30,45,45,70,...,10,10,10,10,10,0,3,2,2,1
1,43808,2,1,2,50,45,40,55,45,55,...,45,55,60,40,55,80,5,1,2,2
2,50607,1,1,2,50,55,50,45,50,55,...,25,20,25,20,45,20,3,1,2,2
